# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.
Notice that `PlaneWaveBasis` defaults to just using the ``Γ``-point
if no ``k``-point options are provided.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5)

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.119326e+02     1.657688e+00
 * time: 0.09101414680480957
     1     1.120210e+01     1.021875e+00
 * time: 0.9401390552520752
     2    -1.167959e+01     1.134925e+00
 * time: 1.0767300128936768
     3    -3.384695e+01     7.893461e-01
 * time: 1.2387821674346924
     4    -4.745770e+01     5.541462e-01
 * time: 1.3855221271514893
     5    -5.669431e+01     1.961684e-01
 * time: 1.5296630859375
     6    -5.958016e+01     1.397412e-01
 * time: 1.636789083480835
     7    -6.075797e+01     6.029916e-02
 * time: 1.750338077545166
     8    -6.124929e+01     4.341717e-02
 * time: 1.865368127822876
     9    -6.152061e+01     4.542566e-02
 * time: 1.9740240573883057
    10    -6.174143e+01     2.763963e-02
 * time: 2.0892720222473145
    11    -6.190652e+01     2.172077e-02
 * time: 2.20345401763916
    12    -6.199490e+01     1.770741e-02
 * time: 2.311795949935913
    13    -6.205943e+01     1.663658e-02
 * time: 2.4258739948272705
 

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671073
    AtomicLocal         -18.8557729
    AtomicNonlocal      14.8522685
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485383 
    Xc                  -19.3336824

    total               -62.261666461958
